In [ ]:
import psycopg2
import pandas as pd

In [ ]:
# connexion BDD
host = "localhost"
database = "DB_SUPERSTORE"
user = "postgres"
password = "root"

conn = psycopg2.connect(
    host=host,
    database=database,
    user=user,
    password=password
)

In [ ]:
# Create a cursor
cur = conn.cursor()

# Executing an SQL query
cur.execute("SELECT * FROM s_superstore.tb_superstore")
data = cur.fetchall()

# Transform results into Pandas DataFrame
df = pd.DataFrame(data, columns=["order_id","order_date","ship_date","ship_mode","customer_id","customer_name","segment","country","city","state","postal_code","region","product_id","category","sub_category","product_name","sales","quantity","discount","profit"])

In [ ]:
df['category'].hist()
df['region'].hist()

In [ ]:
# Delete lines with no sales value
df = df.dropna(subset=["quantity"]) #With this modification we now have 9987 rows

# Define the type of each column
df['order_id'] = df['order_id'].astype(str)
df['order_date'] = pd.to_datetime(df['order_date'])
df['ship_date'] = pd.to_datetime(df['ship_date'])
df['ship_mode'] = df['ship_mode'].astype(str)
df['customer_id'] = df['customer_id'].astype(str)
df['customer_name'] = df['customer_name'].astype(str)
df['segment'] = df['segment'].astype(str)
df['country'] = df['country'].astype(str)
df['city'] = df['city'].astype(str)
df['state'] = df['state'].astype(str)
df['postal_code'] = df['postal_code'].astype(str)
df['region'] = df['region'].astype(str)
df['product_id'] = df['product_id'].astype(str)
df['category'] = df['category'].astype(str)
df['sub_category'] = df['sub_category'].astype(str)
df['product_name'] = df['product_name'].astype(str)
df['sales'] = df['sales'].astype(float)
df['quantity'] = df['quantity'].astype(int)
df['discount'] = df['discount'].astype(float)
df['profit'] = df['profit'].astype(float)

In [ ]:
# Identify products with duplicate IDs
duplicate_product_id = df.groupby('product_id')['product_name'].nunique()
products_with_duplicates = duplicate_product_id[duplicate_product_id > 1].index #List product_id with two names

# Update duplicate product IDs
for product_id in products_with_duplicates:
    # Retrieve all lines with this product ID
    rows_with_duplicate_id = df.loc[df['product_id'] == product_id]
    
    # Identify lines with second product name
    second_product_names = rows_with_duplicate_id['product_name'].value_counts().index[1:]
    
    # Update line IDs with second product name
    for product_name in second_product_names:
        rows_to_update = rows_with_duplicate_id.loc[rows_with_duplicate_id['product_name'] == product_name]
        new_product_id = f"{product_id}2"
        for i, row in rows_to_update.iterrows():
            df.at[i, 'product_id'] = new_product_id # Now we have 1893 different products

# Calculate the unit price using the formula: sales / (quantity * (1 - discount))
df['unit_price'] = (df['sales'] / (df['quantity'] * (1 - df['discount']))).round(2)

# Identify products with the same name
# List products with the same name but different IDs and different category or sub cat)
diff_id = df.loc[df.groupby('product_name')['product_id'].transform('nunique') == 2].copy()

#We use == 2 because after analysis we noticed that only those with 2 IDs are identical.
diff_id = diff_id[diff_id.groupby('product_name')['unit_price'].transform('nunique') == 1]
df.loc[diff_id.index, 'product_id'] = diff_id.groupby('product_name')['product_id'].transform('first')

df['profit']=df['profit'].round(2)

#df.describe()

In [ ]:
# Product creation
list = ['product_id','product_name', 'category', 'sub_category', 'unit_price']
product = df[list]

# Delete duplicate product_ids to have unique values in the dimension
product = product.drop_duplicates(subset="product_id")
#product.describe() # We find ourselves with 1,886 different products

product['product_id'] = df['product_id'].astype(str)
product['category'] = df['category'].astype(str)
product['sub_category'] = df['sub_category'].astype(str)
product['product_name'] = df['product_name'].astype(str)

In [ ]:
# customer creation
list = ['customer_id','customer_name', 'segment','region', 'state', 'city', 'postal_code']
customer = df[list]

# Delete duplicate values
customer = customer.drop_duplicates(subset="customer_id")
#customer.describe()

In [ ]:
# Order Creation
to_drop = ['customer_name', 'segment', 'country', 'city', 'state', 'postal_code','region', 'category', 'sub_category', 'product_name', 'unit_price']
order = df.drop(columns=to_drop)

In [ ]:
# Insert data from the dataframe into the s_superstore.tb_customer table
cursor = conn.cursor()
for index, row in customer.iterrows():
    sql = "INSERT INTO s_superstore.tb_customer (customer_id, customer_name, segment, region, state, city, postal_code) VALUES (%s, %s, %s, %s, %s, %s, %s)"
    values = (row['customer_id'], row['customer_name'], row['segment'], row['region'], row['state'], row['city'], row['postal_code'])
    cursor.execute(sql, values)

conn.commit()

In [ ]:
# Insert data from the dataframe into the s_superstore.tb_product table
cursor = conn.cursor()
for index, row in product.iterrows():
    sql = "INSERT INTO s_superstore.tb_product (product_id, product_name, category, sub_category, unit_price) VALUES (%s, %s, %s, %s, %s)"
    values = (row['product_id'], row['product_name'], row['category'], row['sub_category'], row['unit_price'])
    cursor.execute(sql, values)

conn.commit()

In [ ]:
# Insert data from the dataframe into the s_superstore.tb_order table
cursor = conn.cursor()
for index, row in order.iterrows():
    sql = "INSERT INTO s_superstore.tb_order (order_id, order_date, ship_date, ship_mode, customer_id, product_id, sales, quantity, discount, profit) VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)"
    values = (row['order_id'], row['order_date'], row['ship_date'],row['ship_mode'], row['customer_id'], row['product_id'], row['sales'], row['quantity'], row['discount'], row['profit'])
    cursor.execute(sql, values)

conn.commit()
cursor.close()
conn.close()